# Stochastic Differential Equations: Lab 2

In [ ]:
from IPython.core.display import HTML
css_file = 'https://raw.githubusercontent.com/ngcm/training-public/master/ipython_notebook_styles/ngcmstyle.css'
HTML(url=css_file)

This background for these exercises is article of D Higham, [*An Algorithmic Introduction to Numerical Simulation of Stochastic Differential Equations*, SIAM Review 43:525-546 (2001)](http://epubs.siam.org/doi/abs/10.1137/S0036144500378302).
Higham provides Matlab codes illustrating the basic ideas at <http://personal.strath.ac.uk/d.j.higham/algfiles.html>, which are also given in the paper.

For random processes in `python` you should look at the `numpy.random` module. To set the initial seed (which you should *not* do in a real simulation, but allows for reproducible testing), see `numpy.random.seed`.

## Milstein's method

In [ ]:
%matplotlib inline
import numpy
from matplotlib import pyplot
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16
rcParams['figure.figsize'] = (12,6)

Apply Milstein's method to the stochastic differential equation

$$
\begin{equation}
  dX(t) = \lambda X(t) + \mu X(t) dW(t), \qquad X(0) = X_0.
\end{equation}
$$

Choose any reasonable values of the free parameters $\lambda, \mu, X_0$.

The exact solution to this equation is $X(t) = X(0) \exp \left[ \left( \lambda - \tfrac{1}{2} \mu^2 \right) t + \mu W(t) \right]$. Fix the timetstep and compare your solution to the exact solution.

Check the convergence again.

Compare the *performance* of the Euler-Maruyama and Milstein method using eg `timeit`. At what point is one method better than the other?

## Population problem

Apply the algorithms, convergence and performance tests to the SDE

$$
\begin{equation}
  dX(t) = r X(t) (K - X(t)) dt + \beta X(t) dW(t), \qquad X(0) = X_0.
\end{equation}
$$

Use the parameters $r = 2, K = 1, \beta = 0.25, X_0 = 0.5$.

Investigate how the behaviour varies as you change the parameters $r, K, \beta$.